# LightEvalを使用してLLMを評価および分析する方法

このノートブックでは、LightEvalを使用して大規模言語モデル（LLM）を評価および分析する方法を示します。LightEvalは、さまざまなタスクにわたってモデルの性能を評価するための柔軟で使いやすいフレームワークです。

## 1. 開発環境のセットアップ

最初のステップは、必要なライブラリをインストールすることです。`lighteval`、`transformers`、`datasets`を含みます。


In [ ]:
# Google Colabでの要件のインストール
# !pip install lighteval transformers datasets

# Hugging Faceへの認証

from huggingface_hub import login

login()

# 便利のため、Hugging Faceのトークンを環境変数HF_TOKENとして設定できます

## 2. モデルとデータセットの読み込み

In [13]:
# モデルとデータセットの読み込み
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# サンプルデータセットの読み込み
dataset = load_dataset("lighteval", "mmlu")
dataset

## 3. LightEvalを使用した評価

LightEvalを使用してモデルを評価するために、評価タスクを定義し、パイプラインを設定します。

In [ ]:
# LightEvalのタスクとパイプラインをインポート
from lighteval.tasks import Task, Pipeline
from lighteval.metrics import EvaluationTracker

# 評価するタスクを定義
tasks = [
    "mmlu|anatomy|0|0",
    "mmlu|high_school_biology|0|0",
    "mmlu|high_school_chemistry|0|0",
    "mmlu|professional_medicine|0|0"
]

# パイプラインパラメータを設定
pipeline_params = {
    "max_samples": 40,  # 評価するサンプル数
    "batch_size": 1,    # 推論のバッチサイズ
    "num_workers": 4    # ワーカープロセスの数
}

# 評価トラッカーを作成
evaluation_tracker = EvaluationTracker(
    output_path="./results",
    save_generations=True
)

# パイプラインを作成
pipeline = Pipeline(
    tasks=tasks,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model=model
)

# 評価を実行
pipeline.evaluate()

# 結果を取得して表示
results = pipeline.get_results()
pipeline.show_results()

## 4. 結果の分析

評価結果を分析し、モデルの性能を理解します。

In [ ]:
# pandasを使用して結果を分析
import pandas as pd

# 結果をDataFrameに変換
df = pd.DataFrame(results)

# 結果を表示
print(df)

# 結果を視覚化
df.plot(kind="bar", x="Task", y="Value", legend=False)
plt.ylabel("Accuracy")
plt.title("Model Performance on Different Tasks")
plt.show()